In [1]:
import sys,os
project_root = os.path.dirname(os.getcwd())
sys.path.insert(0,project_root)

from src.data_handling.load_data import *
from src.data_handling.preprocessing import *
from src.models.model_setup import *
from src.trainer.training import *
from src.trainer.file_utils import *

c:\Users\Hector Auvinen\Documents\GitHub\adapter_experiments\adapter_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
task = "sst2"
model_name = "bert-base-uncased"
num_labels = 2
output_dir = "C:/Users/Hector Auvinen/Desktop/eval_results"

In [3]:
data = load_hf_dataset(task,debug=False)

tokenizer = get_tokenizer(model_name)

encode = get_encoding(task)

Using the latest cached version of the dataset since glue couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'sst2' at C:\Users\Hector Auvinen\.cache\huggingface\datasets\glue\sst2\0.0.0\bcdcba79d07bc864c1c254ccfcedcce55bcc9a8c (last modified on Thu Feb  1 17:17:12 2024).


using bert tokenizer
getting encoding:
<function encode_sst2 at 0x000001CBCB3B0160>


In [4]:
#def preprocess_dataset(dataset,encoding_func,tokenizer):
dataset = preprocess_dataset(data,encode,tokenizer)

mapped
{'train': ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'], 'validation': ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'], 'test': ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask']}


In [6]:
model = setup_model(model_name,num_labels)

adapter_config = adapters.BnConfig(
                          output_adapter=True,
                          mh_adapter=False,
                          reduction_factor=2,
                          non_linearity="relu")

model = add_clf_adapter(task_name=task,model=model,num_labels=num_labels,adapter_config=adapter_config)

default_args = TrainingParameters()
default_args.lr_scheduler_type = "linear"


Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_args = get_training_arguments(default_args)

trainer = get_trainer(train_args,dataset,model,early_stopping=3)

trainer.train()

                                                 
  1%|          | 50/8419 [00:22<29:31,  4.72it/s]

{'eval_loss': 0.5497307181358337, 'eval_accuracy': 0.694954128440367, 'eval_runtime': 12.0503, 'eval_samples_per_second': 72.363, 'eval_steps_per_second': 9.045, 'epoch': 0.01}


                                                   
  1%|          | 100/8419 [00:48<35:15,  3.93it/s]

{'eval_loss': 0.39171481132507324, 'eval_accuracy': 0.8532110091743119, 'eval_runtime': 13.2594, 'eval_samples_per_second': 65.765, 'eval_steps_per_second': 8.221, 'epoch': 0.01}


                                                    
  2%|▏         | 150/8419 [01:12<32:08,  4.29it/s]

...

                                                    
  9%|▉         | 750/8419 [06:16<31:30,  4.06it/s]

{'eval_loss': 0.2661172151565552, 'eval_accuracy': 0.8979357798165137, 'eval_runtime': 12.2229, 'eval_samples_per_second': 71.341, 'eval_steps_per_second': 8.918, 'epoch': 0.09}


  9%|▉         | 750/8419 [06:16<1:04:11,  1.99it/s]

{'train_runtime': 376.6543, 'train_samples_per_second': 178.809, 'train_steps_per_second': 22.352, 'train_loss': 0.3673608093261719, 'epoch': 0.09}


TrainOutput(global_step=750, training_loss=0.3673608093261719, metrics={'train_runtime': 376.6543, 'train_samples_per_second': 178.809, 'train_steps_per_second': 22.352, 'train_loss': 0.3673608093261719, 'epoch': 0.09})

In [7]:
eval_results = trainer.evaluate()

100%|██████████| 109/109 [00:11<00:00,  9.62it/s]


In [8]:
write_eval_results(eval_results,output_dir,task,trainer,adapter_config)

Writing eval results
{'eval_loss': 0.2526504099369049, 'eval_accuracy': 0.8990825688073395, 'eval_runtime': 11.4522, 'eval_samples_per_second': 76.143, 'eval_steps_per_second': 9.518, 'epoch': 0.08}


In [6]:
"""for attr_name, attr_value in vars(default_args).items():
    print(f"{attr_name}: {attr_value}")"""
"""
    train_args = {"label_names":["labels"],
    "evaluation_strategy":"steps",
    "learning_rate":1e-4,
    "num_train_epochs":1,
    "per_device_train_batch_size":8,
    "per_device_eval_batch_size":8,
    "eval_steps":50,
    "logging_steps":200,
    "output_dir":"/eval_results",
    "overwrite_output_dir":True,
    "remove_unused_columns":False,
    "lr_scheduler_type":'linear',
    "load_best_model_at_end":True,
    "metric_for_best_model" : "accuracy",
    "early_stopping_patience":3,
    "save_total_limit":5
}
"""

'for attr_name, attr_value in vars(default_args).items():\n    print(f"{attr_name}: {attr_value}")'